In [4]:
!pip install -q pyautogen

In [5]:
pip install -U langchain-community

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install langchain openai faiss-cpu pypdf unstructured chromadb

  Using cached openai-1.77.0-py3-none-any.whl.metadata (25 kB)
  Using cached faiss_cpu-1.11.0-cp312-cp312-win_amd64.whl.metadata (5.0 kB)
  Using cached pypdf-5.4.0-py3-none-any.whl.metadata (7.3 kB)
  Using cached unstructured-0.17.2-py3-none-any.whl.metadata (24 kB)
  Using cached chromadb-1.0.8-cp39-abi3-win_amd64.whl.metadata (7.0 kB)
  Using cached jiter-0.9.0-cp312-cp312-win_amd64.whl.metadata (5.3 kB)
  Using cached filetype-1.2.0-py2.py3-none-any.whl.metadata (6.5 kB)
  Using cached python_magic-0.4.27-py2.py3-none-any.whl.metadata (5.8 kB)
  Using cached emoji-2.14.1-py3-none-any.whl.metadata (5.7 kB)
  Using cached python_iso639-2025.2.18-py3-none-any.whl.metadata (14 kB)
  Using cached langdetect-1.0.9.tar.gz (981 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached rapidfuzz-3.13.0-cp312-cp312-win_amd64.whl.metadata (12 kB)
  Using cached backoff-2.2.1-py3-none-any.whl.metadata (14 kB)
  Using cached unstr

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.32.0 requires protobuf<5,>=3.20, but you have protobuf 5.29.4 which is incompatible.
streamlit 1.32.0 requires tenacity<9,>=8.1.0, but you have tenacity 9.1.2 which is incompatible.


In [9]:
pdf_folder = "./Sample_Files"
xlsx_folder = "./Sample_Files"

In [10]:
import os
os.environ["OPENAI_API_KEY"]="" # Replace with your own API key

In [11]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

In [13]:
%%writefile OAI_CONFIG_LIST
[
    {
        "model": "gpt-4",
        "api_key": "",
        "tags": ["gpt-4", "tool"]
    }
]


Writing OAI_CONFIG_LIST


In [14]:
# Load PDF files
from langchain.document_loaders import PyPDFLoader, UnstructuredExcelLoader

all_documents = []
for filename in os.listdir(pdf_folder):
    if filename.lower().endswith(".pdf"):
        file_path = os.path.join(pdf_folder, filename)
        loader = PyPDFLoader(file_path)
        documents = loader.load()
        all_documents.extend(documents)


In [15]:
# Chunking the text
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
chunks = splitter.split_documents(all_documents)


In [16]:
# Vectorization
embeddings = OpenAIEmbeddings()

vectorstore = FAISS.from_documents(chunks, embeddings)
vectorstore.save_local("rag_index")

C:\Users\11944\AppData\Local\Temp\ipykernel_29408\3859304846.py:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


In [17]:
# Set up Retriever
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k":5})

llm = ChatOpenAI(model_name="gpt-4")

rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever
)

C:\Users\11944\AppData\Local\Temp\ipykernel_29408\1985165244.py:4: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-4")


In [18]:
query_list = [
    'Did the company beat, meet, or miss its sales expectations this quarter?',
    "Did the company's sales increase year-over-year (YoY)?",
    "Did the company beat, meet, or miss its earnings expectations this quarter?",
    "Did the company's earnings increase year-over-year (YoY)?",
    "Did the company raise or lower its sales guidance?",
    "Did the company raise or lower its earnings guidance?",
    "What is the company's outlook on future sales growth? (Up, Flat, or Down)",
    "What is the company's guidance on future earnings? (Up, Flat, or Down)"
]
result_dic = {}
result_list = []

In [19]:
for query in query_list:
  result = rag_chain.run(query)
  result_dic[query] = result
  result_list.append(result)

C:\Users\11944\AppData\Local\Temp\ipykernel_29408\3022571715.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = rag_chain.run(query)


In [20]:
for i in result_dic:
  print(i,':',result_dic[i])
  print('\n')

Did the company beat, meet, or miss its sales expectations this quarter? : The text does not provide specific information on whether the company beat, met, or missed its sales expectations for this quarter.


Did the company's sales increase year-over-year (YoY)? : Yes, the company's sales increased year-over-year. The net sales increased $31.9 billion or 5.0% for fiscal 2025 and $36.8 billion or 6.1% for fiscal 2024 when compared to the previous fiscal year.


Did the company beat, meet, or miss its earnings expectations this quarter? : The text does not provide specific information on whether the company beat, met, or missed its earnings expectations for the quarter.


Did the company's earnings increase year-over-year (YoY)? : Yes, the company's earnings did increase year-over-year. Adjusted operating income increased by 9.4%, and consolidated revenue grew by 5.6% in constant currency.


Did the company raise or lower its sales guidance? : The text does not provide information on wh

In [ ]:
query = "What is the sales number of Q4 FYq5"
result = rag_chain.run(query)
print("\n💬 回答：")
print(result)

<ipython-input-21-3f086cf3ccc6>:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = rag_chain.run(query)



💬 回答：
Q4 FY25的销售额为$36.4 billion。
